# Content Base Recommender

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data=pd.read_csv('Coursera.csv')
df=data.copy()

In [ ]:
df.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [ ]:
# null values count
df.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

In [ ]:
df=df[['Course Name','Difficulty Level','Course Description','Skills']]

## preprocessing

In [ ]:
#removing unnecessary punctuation's
def remove_punctuation(txt):
  txt_nopunt= "".join([c for c in txt if c not in string.punctuation])
  return txt_nopunt

In [ ]:
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['Course Description'] = df['Course Description'].str.replace('http\S+|www.\S+', '', case=False)

<ipython-input-55-2ddd663ba56d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Course Description'] = df['Course Description'].str.replace('http\S+|www.\S+', '', case=False)


In [ ]:
df['Course Description']=df['Course Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Course Name']=df['Course Name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Skills']=df['Skills'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df['Course Name']=df['Course Name'].replace(","," ",regex=True)
df['Course Description']=df['Course Description'].replace(","," ",regex=True)
df['Skills']=df['Skills'].replace(","," ",regex=True)
df['Skills']=df['Skills'].replace("-"," ",regex=True)

df['Course Name']=df['Course Name'].apply(lambda x: remove_punctuation(x))
df['Course Description']=df['Course Description'].apply(lambda x: remove_punctuation(x))
df['Skills']=df['Skills'].apply(lambda x: remove_punctuation(x))

df['Course Description']=df['Course Description'].replace("�","'",regex=True)

df['tags']=df['Course Name']+' '+ df['Difficulty Level']+' '+ df['Course Description']+' '+df['Skills']
new_df=df[['Course Name','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

<ipython-input-57-8b60b591a07f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

<ipython-input-58-0e0bbe40f26f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [ ]:
new_df['tags'][0]


'write feature length screenplay film television beginner write full length feature film script course write complete featurelength screenplay film television serious drama romantic comedy anything learn break creative process components discover structured process allows produce polished pitchready script end course completing project increase confidence ideas abilities feel prepared pitch first script get started next course designed tap creativity based active learning actual learning takes place within activities writing learn link trailer course view trailer please copy paste link browser learner review love approach professor wheeler takes towards course point easy follow informative would definitely recommend anyone interested taking screenplay writing course course curriculum simple adopt professional writers room process write post work peer review share feedback peers revise work feedback receive peers thats real world feel professional writers room yet prior experience write

Stemming

We use stemming to extract the base form of the words by removing affixes from them.

In [ ]:
ps=PorterStemmer()

In [ ]:
def stem(text):
  y=[]

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

<ipython-input-62-402dd7cb6b10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [ ]:
new_df['tags'][0]

'write featur length screenplay film televis beginn write full length featur film script cours write complet featurelength screenplay film televis seriou drama romant comedi anyth learn break creativ process compon discov structur process allow produc polish pitchreadi script end cours complet project increas confid idea abil feel prepar pitch first script get start next cours design tap creativ base activ learn actual learn take place within activ write learn link trailer cours view trailer pleas copi past link browser learner review love approach professor wheeler take toward cours point easi follow inform would definit recommend anyon interest take screenplay write cours cours curriculum simpl adopt profession writer room process write post work peer review share feedback peer revis work feedback receiv peer that real world feel profession writer room yet prior experi writer requir im propon experienti learn activ learn lectur short sometim two minut long point design stepbystep pro

## Text Vectorization

In [ ]:
cv=CountVectorizer()

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

## similarity measure

In [ ]:
#using cosine similariy btw the tags
similarity=cosine_similarity(vectors)
pd.DataFrame(similarity)

,0,1,2,3,4,5,6,7,8,9,...,3512,3513,3514,3515,3516,3517,3518,3519,3520,3521
0,1.000000,0.038258,0.094104,0.074150,0.128492,0.131863,0.064421,0.203068,0.162800,0.253142,...,0.121886,0.163697,0.090035,0.068720,0.196853,0.067323,0.057543,0.082099,0.061370,0.082665
1,0.038258,1.000000,0.018156,0.114605,0.074473,0.064322,0.570034,0.023437,0.074494,0.129327,...,0.144193,0.041329,0.013471,0.013880,0.050318,0.090652,0.016271,0.284268,0.012396,0.085623
2,0.094104,0.018156,1.000000,0.026010,0.012676,0.020072,0.003094,0.058511,0.033417,0.067575,...,0.024144,0.051588,0.010089,0.043314,0.033647,0.028289,0.011606,0.050268,0.105728,0.053439
3,0.074150,0.114605,0.026010,1.000000,0.093755,0.040314,0.078111,0.051890,0.080038,0.086172,...,0.146107,0.071048,0.027016,0.064623,0.072084,0.051946,0.013320,0.128957,0.011838,0.147193
4,0.128492,0.074473,0.012676,0.093755,1.000000,0.077343,0.052296,0.160000,0.078016,0.171722,...,0.390688,0.136796,0.045144,0.060298,0.316738,0.365685,0.040393,0.167598,0.030771,0.157290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,0.067323,0.090652,0.028289,0.051946,0.365685,0.060132,0.105034,0.164660,0.043525,0.082475,...,0.539194,0.054950,0.010074,0.060553,0.255358,1.000000,0.014487,0.147637,0.046352,0.089650
3518,0.057543,0.016271,0.011606,0.013320,0.040393,0.012849,0.015843,0.062652,0.099702,0.071135,...,0.020233,0.059883,0.079219,0.026618,0.028719,0.014487,1.000000,0.033314,0.031695,0.050355
3519,0.082099,0.284268,0.050268,0.128957,0.167598,0.073330,0.297077,0.083281,0.060661,0.121476,...,0.247436,0.086156,0.070203,0.067816,0.191994,0.147637,0.033314,1.000000,0.005383,0.053548
3520,0.061370,0.012396,0.105728,0.011838,0.030771,0.027408,0.000000,0.140418,0.071419,0.105940,...,0.035965,0.106444,0.015306,0.027600,0.057174,0.046352,0.031695,0.005383,1.000000,0.085615


In [ ]:
display(new_df)

,Course Name,tags
0,Write A Feature Length Screenplay For Film Or ...,write featur length screenplay film televis be...
1,Business Strategy Business Model Canvas Analys...,busi strategi busi model canva analysi miro be...
2,Silicon Thin Film Solar Cells,silicon thin film solar cell advanc cours cons...
3,Finance Managers,financ manag intermedi come number alway meet ...
4,Retrieve Data using SingleTable SQL Queries,retriev data use singlet sql queri beginn cour...
...,...,...
3517,Capstone Retrieving Processing Visualizing D...,capston retriev process visual data python beg...
3518,Patrick Henry Forgotten Founder,patrick henri forgotten founder intermedi 'giv...
3519,Business intelligence data analytics Generate ...,busi intellig data analyt gener insight advanc...
3520,Rigid Body Dynamics,rigid bodi dynam beginn cours teach dynam one ...


In [ ]:
def recommend(course):
  course_index=new_df[new_df['Course Name']==course].first_valid_index()
  distances=similarity[course_index]
  dict={}
  for i in range(0,len(new_df['tags'])):
    dict[i]=distances[i]
  dict=sorted(dict.items(),key=lambda x:x[1], reverse=True)
  for i in range(1,7):
    index=dict[i][0]
    print(new_df['Course Name'][index])

In [ ]:
#recommending using Text Vecorization
recommend('Silicon Thin Film Solar Cells')

Physics silicon solar cells
Introduction solar cells
Organic Solar Cells  Theory Practice
Photovoltaic solar energy
Solar Energy Systems Overview
Solar Energy Basics


## tf-idf

In [ ]:
tfidf=TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(df['tags'])

In [ ]:
similarity=cosine_similarity(tfidf_matrix)

In [ ]:
#recommendation using tfidf
recommend('Silicon Thin Film Solar Cells')

Physics silicon solar cells
Introduction solar cells
Organic Solar Cells  Theory Practice
Photovoltaic solar energy
Solar Energy Systems Overview
Solar Energy Basics


# Collaborative Based recommender

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

from collections import defaultdict

In [ ]:
rev_df=pd.read_csv('Coursera_reviews.csv')

In [ ]:
rev_df.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [ ]:
rev_df=rev_df[['reviewers','course_id','rating']]
rev_df.head()

,reviewers,course_id,rating
0,By Robert S,google-cbrs-cpi-training,4
1,By Gabriel E R,google-cbrs-cpi-training,4
2,By Jacob D,google-cbrs-cpi-training,4
3,By Dale B,google-cbrs-cpi-training,4
4,By Sean G,google-cbrs-cpi-training,4


In [ ]:
rev_df.isnull().sum()

reviewers    0
course_id    0
rating       0
dtype: int64

In [ ]:
reviewers=rev_df['reviewers'].unique()
reviewers

array(['By Robert S', 'By Gabriel E R', 'By Jacob D', ...,
       'By Juan C S M', 'By Erlinton R A', 'By Kasirivu A K'],
      dtype=object)

In [ ]:
reviewers=reviewers[:270000]

In [ ]:
rev_matrix=rev_df[~rev_df['reviewers'].isin(reviewers)]

In [ ]:
rev_matrix=rev_df.pivot_table(index=['reviewers'],columns=['course_id'],values='rating').fillna(0)

In [ ]:
rev_matrix=rev_df.reset_index().melt(id_vars=['reviewers'])
rev_matrix

,reviewers,variable,value
0,By Robert S,index,0
1,By Gabriel E R,index,1
2,By Jacob D,index,2
3,By Dale B,index,3
4,By Sean G,index,4
...,...,...,...
4364128,By Brijesh K,rating,5
4364129,By Vasavi V M,rating,5
4364130,By Drishti D,rating,5
4364131,By FAUSTINE F K,rating,5


In [ ]:
reader = Reader(rating_scale=(0, 5))
data= Dataset.load_from_df(rev_df, reader=reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [ ]:
trainset,testset = train_test_split(data,test_size=0.25)

### co-clustering

In [ ]:
cocul_model=CoClustering()
cocul_model.fit(trainset)
cross_validate(cocul_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4963  0.4960  0.4947  0.4916  0.5025  0.4962  0.0036  
MAE (testset)     0.2441  0.2462  0.2472  0.2391  0.2581  0.2469  0.0063  
Fit time          102.28  94.61   104.84  95.28   100.25  99.45   3.96    
Test time         4.95    5.59    2.06    2.01    2.85    3.49    1.50    


{'test_rmse': array([0.49626001, 0.49603044, 0.49467219, 0.49161911, 0.50250184]),
 'test_mae': array([0.24410927, 0.24615738, 0.24715509, 0.23912349, 0.25814473]),
 'fit_time': (102.27842378616333,
  94.61328315734863,
  104.83794283866882,
  95.27727031707764,
  100.25268864631653),
 'test_time': (4.951104640960693,
  5.59474515914917,
  2.057431936264038,
  2.014998197555542,
  2.8453528881073)}

In [ ]:
prediction1=cocul_model.test(testset)

In [ ]:
pred1=pd.DataFrame(prediction1)
pred1

,uid,iid,r_ui,est,details
0,By A. S,open-source-tools-for-data-science,2.0,3.608064,{'was_impossible': False}
1,By Ashley C,childnutrition,3.0,4.566720,{'was_impossible': False}
2,By Edward S,introduction-to-computer-programming,4.0,3.808410,{'was_impossible': False}
3,By Saiaf B R,wind-energy,5.0,4.881182,{'was_impossible': False}
4,By Dmitriy K,technical-support-fundamentals,5.0,4.909898,{'was_impossible': False}
...,...,...,...,...,...
363673,By Lisset S R T,mafash,5.0,4.951074,{'was_impossible': False}
363674,By Yu Q,deep-neural-network,5.0,5.000000,{'was_impossible': False}
363675,By Kobie T,technical-support-fundamentals,5.0,4.934550,{'was_impossible': False}
363676,By Alejandro D O,mcmc-bayesian-statistics,5.0,5.000000,{'was_impossible': False}


###SVD

In [ ]:
svd_model=SVD()
svd_model.fit(trainset)
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4502  0.4448  0.4472  0.4467  0.4487  0.4475  0.0018  
MAE (testset)     0.2812  0.2772  0.2789  0.2796  0.2796  0.2793  0.0013  
Fit time          30.77   32.70   42.68   34.55   34.05   34.95   4.08    
Test time         3.37    2.80    7.98    5.40    6.04    5.12    1.87    


{'test_rmse': array([0.45017368, 0.44480623, 0.44716326, 0.44673393, 0.44868229]),
 'test_mae': array([0.28120831, 0.27719848, 0.2789127 , 0.27963257, 0.27956681]),
 'fit_time': (30.771774768829346,
  32.70173144340515,
  42.6798837184906,
  34.5539288520813,
  34.04656386375427),
 'test_time': (3.373138666152954,
  2.8008971214294434,
  7.980547904968262,
  5.402459383010864,
  6.0425074100494385)}

In [ ]:
prediction2=svd_model.test(testset)


In [ ]:
pred2=pd.DataFrame(prediction2)
pred2

,uid,iid,r_ui,est,details
0,By A. S,open-source-tools-for-data-science,2.0,2.133038,{'was_impossible': False}
1,By Ashley C,childnutrition,3.0,4.287354,{'was_impossible': False}
2,By Edward S,introduction-to-computer-programming,4.0,4.062721,{'was_impossible': False}
3,By Saiaf B R,wind-energy,5.0,4.938536,{'was_impossible': False}
4,By Dmitriy K,technical-support-fundamentals,5.0,4.950208,{'was_impossible': False}
...,...,...,...,...,...
363673,By Lisset S R T,mafash,5.0,4.858427,{'was_impossible': False}
363674,By Yu Q,deep-neural-network,5.0,4.851241,{'was_impossible': False}
363675,By Kobie T,technical-support-fundamentals,5.0,4.875407,{'was_impossible': False}
363676,By Alejandro D O,mcmc-bayesian-statistics,5.0,5.000000,{'was_impossible': False}


### recommendation

In [ ]:
top_n = defaultdict(list)
n=3
for uid, iid, true_r, est, _ in prediction1:
  top_n[uid].append((iid, est))

for uid, user_ratings in top_n.items():
  user_ratings.sort(key=lambda x: x[1], reverse=True)
  top_n[uid] = user_ratings[:n]

for uid, user_ratings in top_n.items():
  print(uid, user_ratings)

Streaming output truncated to the last 5000 lines.
By ANGEL F F [('introduction-cybersecurity-cyber-attacks', 3.5586762051200393)]
By shem m a [('programming-fundamentals', 4.956633242824712)]
By Basem B S H [('introduction-to-computer-programming', 4.220695024623889)]
By Rathsara R [('what-is-a-proof', 4.701790464004516)]
By SELVADEVI H [('python-crash-course', 5)]
By hilary [('introclassicalmusic', 5)]
By Anshul M [('neural-networks-deep-learning', 4.577454613975173)]
By Siddharth S G [('machine-learning', 5)]
By Alma  A R F [('learning-how-to-learn', 5)]
By Chuin C L [('python-network-data', 4.984034368456347)]
By Viktor S O [('progfun1', 4.958524893966216)]
By Marianne D [('uva-darden-design-thinking-innovation', 4.2734421502174085)]
By Kanisk U [('ai-for-medical-diagnosis', 5)]
By PARAS M [('company-future-management', 4.696719881694735)]
By Pramila S [('norms', 4.89313194774381)]
By Addagalla S [('computer-networking', 4.867872909167349)]
By Patricia F G M [('learn-korean', 5)]
B

In [ ]:
top_n = defaultdict(list)
n=3
for uid, iid, true_r, est, _ in prediction2:
  top_n[uid].append((iid, est))

for uid, user_ratings in top_n.items():
  user_ratings.sort(key=lambda x: x[1], reverse=True)
  top_n[uid] = user_ratings[:n]

for uid, user_ratings in top_n.items():
  print(uid, user_ratings)

Streaming output truncated to the last 5000 lines.
By ANGEL F F [('introduction-cybersecurity-cyber-attacks', 4.041369032568594)]
By shem m a [('programming-fundamentals', 5)]
By Basem B S H [('introduction-to-computer-programming', 4.86841200356238)]
By Rathsara R [('what-is-a-proof', 4.9561486561807016)]
By SELVADEVI H [('python-crash-course', 4.817472525650271)]
By hilary [('introclassicalmusic', 5)]
By Anshul M [('neural-networks-deep-learning', 4.619178259776129)]
By Siddharth S G [('machine-learning', 4.7007969657130015)]
By Alma  A R F [('learning-how-to-learn', 4.947151335439206)]
By Chuin C L [('python-network-data', 4.974898269616178)]
By Viktor S O [('progfun1', 4.945809596460634)]
By Marianne D [('uva-darden-design-thinking-innovation', 4.388028837623108)]
By Kanisk U [('ai-for-medical-diagnosis', 4.832381309243597)]
By PARAS M [('company-future-management', 4.801035818374466)]
By Pramila S [('norms', 4.746164912717763)]
By Addagalla S [('computer-networking', 4.65908466275

We can conclude that SVD model recommendation is better than the coclustering model.